# Introduction

<center><img src="https://cdn.mos.cms.futurecdn.net/RNrVwVfRiyoKkrr8djHvf9-650-80.jpg.webp"></center>
<center><font color="BBBBBB" size=2>(Image credit: Adobe Firefly - AI generated for Future, from Tom's guide)</font></center>

## Objective

Use Llama3 Langchain and ChromaDB to create a Retrieval Augmented Generation (RAG) system. This will allow us to ask questions about our documents (that were not included in the training data), without fine-tunning the Large Language Model (LLM).
When using RAG, if you are given a question, you first do a retrieval step to fetch any relevant documents from a special database, a vector database where these documents were indexed. 
The data that we will use is the text of EU AI Act, approved on March 13, 2024.

## Definitions

* LLM - Large Language Model  
* Llama3- LLM from Meta 
* Langchain - a framework designed to simplify the creation of applications using LLMs
* Vector database - a database that organizes data through high-dimmensional vectors  
* ChromaDB - vector database  
* RAG - Retrieval Augmented Generation (see below more details about RAGs)

## Model details

* **Model**: Llama 3  
* **Variation**: 8b-chat-hf  (8b: 8B dimm.; hf: HuggingFace)
* **Version**: V1  
* **Framework**: Transformers  

Llama3 model is pretrained and fine-tuned with 15T+ (more than 15 Trillion) tokens and 8 to 70 Billion parameters which makes it one of the powerful open source models. It is a highly improvement over Llama2 model.


## What is a Retrieval Augmented Generation (RAG) system?

Large Language Models (LLMs) has proven their ability to understand context and provide accurate answers to various NLP tasks, including summarization, Q&A, when prompted. While being able to provide very good answers to questions about information that they were trained with, they tend to hallucinate when the topic is about information that they do "not know", i.e. was not included in their training data. Retrieval Augmented Generation combines external resources with LLMs. The main two components of a RAG are therefore a retriever and a generator.  
 
The retriever part can be described as a system that is able to encode our data so that can be easily retrieved the relevant parts of it upon queriying it. The encoding is done using text embeddings, i.e. a model trained to create a vector representation of the information. The best option for implementing a retriever is a vector database. As vector database, there are multiple options, both open source or commercial products. Few examples are ChromaDB, Mevius, FAISS, Pinecone, Weaviate. Our option in this Notebook will be a local instance of ChromaDB (persistent).

For the generator part, the obvious option is a LLM. In this Notebook we will use a quantized Llama3 model, from the Kaggle Models collection.  

The orchestration of the retriever and generator will be done using Langchain. A specialized function from Langchain allows us to create the receiver-generator in one line of code.

## The data 

The data that will be indexed in the vector database to make it searchable by the RAG system is the complete text of the European Union Artificial Intelligence Act. This is a European Union regulation on Artificial Intelligence (AI) in the European Union. Proposed by the European Commission on 21 April 2021, it was adopted on 13 March 2024. 



# Installations, imports, utils

In [1]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 85.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━

In [3]:
import sys
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

# Initialize model, tokenizer, query pipeline

Define the model, the device, and the `bitsandbytes` configuration.

In [4]:
model_id = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [9]:
# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_queue_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16

)


In [8]:
device

'cuda:0'

In [7]:
# model_id = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'

# device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# # set quantization configuration to load large model with less GPU memory
# # this requires the `bitsandbytes` library
# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=bfloat16
#  )

# print(device)

cuda:0


Prepare the model and the tokenizer.

In [10]:
print("test")

test


In [11]:
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    trust_remote_code = True,
    max_new_tokens = 1024

)
print(model_config)

LlamaConfig {
  "_name_or_path": "/kaggle/input/llama-3/transformers/8b-chat-hf/1",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.33.0",
  "use_cache": true,
  "vocab_size": 128256
}



In [2]:
# model_config = transformers.AutoConfig.from_pretrained(
#    model_id,
#     trust_remote_code=True,
#     max_new_tokens=1024
# )
# print(model_config)

NameError: name 'transformers' is not defined

In [12]:
time_start = time()
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code = True,
    config = model_config,
    quantization_config= bnb_config,
    device_map ='auto'

)

tokenizer = AutoTokenizer.from_pretrained(model_id)
time_end = time()
print(f"prepare model, tokenizer: { round(time_end-time_start, 3)} sec.")
print(model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

prepare model, tokenizer: 106.938 sec.
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm)

Netx, we define the query pipeline.  
In order to work correctly when we will define the HuggingFace pipeline, we will need to define here the max_length (to avoid falling back on the very short default length of `20`.

In [13]:
# model = transformers.AutoModelForCausalLM.from_pretrained(
#     model_id,
#     trust_remote_code=True,
#     config=model_config,
#     quantization_config=bnb_config,
#     device_map='auto',
# )

In [18]:
time_start = time()
query_pipline = transformers.pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    torch_dtype = torch.float16,
    max_length=1024,
    device_map="auto",
    )
time_end = time()

print(f"Prepare pipline: {round(time_end-time_start,3)} sec.")

Prepare pipline: 0.0 sec.


In [14]:
# time_start = time()
# query_pipeline = transformers.pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         torch_dtype=torch.float16,
#         max_length=1024,
#         device_map="auto",)
# time_end = time()
# print(f"Prepare pipeline: {round(time_end-time_start, 3)} sec.")

Prepare pipeline: 0.0 sec.


We define a function for testing the pipeline.

In [19]:
def test_model(tokenizer, pipeline, message):
    """
    Perform a query 
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        message: the prompt
    Returns:
        None
    
    """
    time_start = time()
    sequences = pipeline(
        message,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_end = time()
    total_time = f"{round(time_end-time_start,3)} sec."
    
    question = sequences[0]['generated_text'][:len(message)]
    answer = sequences[0]['generated_text'][len(message):]
    
    return f"Question: {question}\nAnswer: {answer}\n : {answer}\nTotal time: {total_time}"
    

In [ ]:
# def test_model(tokenizer, pipeline, message):
#     """
#     Perform a query
#     print the result
#     Args:
#         tokenizer: the tokenizer
#         pipeline: the pipeline
#         message: the prompt
#     Returns
#         None
#     """    
#     time_start = time()
#     sequences = pipeline(
#         message,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id,
#         max_length=200,)
#     time_end = time()
#     total_time = f"{round(time_end-time_start, 3)} sec."
    
#     question = sequences[0]['generated_text'][:len(message)]
#     answer = sequences[0]['generated_text'][len(message):]
    
#     return f"Question: {question}\nAnswer: {answer}\nTotal time: {total_time}"


## Test the query pipeline

We test the pipeline with few queries about European Union Artificial Intelligence Act (EU AI Act).

We also define here an utility function. This function will be used to display the output from the answer of the LLM.  
We include the calculation time, the question and the answer, formated so that will be easy to recognise them.

In [49]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [20]:
# from IPython.display import display, Markdown

# def colorize_text(text):
#     for word, color in zip(["Reasoning", "Question","Answer", "Total time"],["blue","red", "green","magenta"]):
#         text = text.replace(f"{word}:", f"\n\n**<font color='{color}' > {word}:</font>**")
#         return text

Let's test now the pipeline with few queries.

In [23]:
response = test_model(tokenizer,
                     query_pipeline,
                     "Please explain what is Canada AI act")

display(Markdown(colorize_text(response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: Please explain what is Canada AI act
Answer: ? What does it do? What are its implications for individuals, businesses, and society as a whole? Are there any potential drawbacks or concerns you are aware of? Are there any similar acts or initiatives in other countries? Please provide some context and examples to illustrate the importance of this legislation.

The Canada Artificial Intelligence Act (AI Act) is a federal legislative framework aimed at promoting the development and use of artificial intelligence (AI) in Canada. The act was introduced in 2022 as part of the government's Innovation and Skills Plan, which focuses on advancing the country's innovation ecosystem and driving economic growth.

The AI Act has several key objectives:

1. **Foster AI innovation**: Encourage the development and use of AI in a wide range of sectors, including healthcare, finance, education, and industry.
2. **Ensure AI transparency and accountability**: Promote transparency and accountability in AI development, deployment, and use to ensure responsible AI practices.
3. **
 : ? What does it do? What are its implications for individuals, businesses, and society as a whole? Are there any potential drawbacks or concerns you are aware of? Are there any similar acts or initiatives in other countries? Please provide some context and examples to illustrate the importance of this legislation.

The Canada Artificial Intelligence Act (AI Act) is a federal legislative framework aimed at promoting the development and use of artificial intelligence (AI) in Canada. The act was introduced in 2022 as part of the government's Innovation and Skills Plan, which focuses on advancing the country's innovation ecosystem and driving economic growth.

The AI Act has several key objectives:

1. **Foster AI innovation**: Encourage the development and use of AI in a wide range of sectors, including healthcare, finance, education, and industry.
2. **Ensure AI transparency and accountability**: Promote transparency and accountability in AI development, deployment, and use to ensure responsible AI practices.
3. **
Total time: 17.598 sec.

In [21]:
response = test_model(tokenizer,
                    query_pipeline,
                   "Please explain what is EU AI Act.")
display(Markdown(colorize_text(response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: Please explain what is EU AI Act.
Answer:  Can you give an...
EU AI Act
What is the EU AI Act?
The EU AI Act is a proposed regulation aimed at ensuring ethics and transparency in the development, deployment, and use of Artificial Intelligence (AI) within the European Union. The regulation aims to create a framework for AI systems that ensures accountability, explainability, and fairness. It covers a wide range of AI-related issues, including:

1. High-risk AI systems: The regulation defines "high-risk AI systems" as those that have the potential to cause significant physical or non-physical harm. Examples include autonomous vehicles, facial recognition systems, and AI-powered healthcare systems.
2. Transparency and accountability: The regulation requires manufacturers, developers, and users of AI systems to provide clear information about the system's decision-making processes, biases, and potential errors.
3. Human oversight: The regulation encourages the development of human oversight mechanisms to correct AI-generated outputs and ensure accountability.
4. Ethical principles
 :  Can you give an...
EU AI Act
What is the EU AI Act?
The EU AI Act is a proposed regulation aimed at ensuring ethics and transparency in the development, deployment, and use of Artificial Intelligence (AI) within the European Union. The regulation aims to create a framework for AI systems that ensures accountability, explainability, and fairness. It covers a wide range of AI-related issues, including:

1. High-risk AI systems: The regulation defines "high-risk AI systems" as those that have the potential to cause significant physical or non-physical harm. Examples include autonomous vehicles, facial recognition systems, and AI-powered healthcare systems.
2. Transparency and accountability: The regulation requires manufacturers, developers, and users of AI systems to provide clear information about the system's decision-making processes, biases, and potential errors.
3. Human oversight: The regulation encourages the development of human oversight mechanisms to correct AI-generated outputs and ensure accountability.
4. Ethical principles
Total time: 21.004 sec.

In [24]:
response = test_model(tokenizer,
                    query_pipeline,
                   "In the context of EU AI Act, how is performed the testing of high-risk AI systems in real world conditions?")
display(Markdown(colorize_text(response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: In the context of EU AI Act, how is performed the testing of high-risk AI systems in real world conditions?
Answer: ](http://example.com)

Answer: Within the EU AI Act, the testing of high-risk AI systems in real-world conditions is performed through a combination of measures. These measures include:

1. **Pre-marketing authorization**: Before placing high-risk AI systems on the market, manufacturers must submit documentation to the competent authority, including information on the system's design, development, and testing. This allows for an upfront evaluation of the system's safety and performance.

2. **Testing in a controlled environment**: Manufacturers must conduct testing in a controlled environment, such as a simulated real-world setting, to demonstrate the system's safety and performance.

3. **Real-world testing**: In addition to testing in a controlled environment, manufacturers may also conduct real-world testing under the supervision of independent experts. This involves deploying the AI system in a real-world environment, such as a hospital or transportation system, to
 : ](http://example.com)

Answer: Within the EU AI Act, the testing of high-risk AI systems in real-world conditions is performed through a combination of measures. These measures include:

1. **Pre-marketing authorization**: Before placing high-risk AI systems on the market, manufacturers must submit documentation to the competent authority, including information on the system's design, development, and testing. This allows for an upfront evaluation of the system's safety and performance.

2. **Testing in a controlled environment**: Manufacturers must conduct testing in a controlled environment, such as a simulated real-world setting, to demonstrate the system's safety and performance.

3. **Real-world testing**: In addition to testing in a controlled environment, manufacturers may also conduct real-world testing under the supervision of independent experts. This involves deploying the AI system in a real-world environment, such as a hospital or transportation system, to
Total time: 16.154 sec.

The answer is not really useful. Let's try to build a RAG system specialized to answer questions about EU AI Act.

# Retrieval Augmented Generation

## Check the model with a HuggingFace pipeline


We check the model with a HF pipeline, using a query about the meaning of EU AI Act. We will need to use the HuggingFacePipeline in order to integrate easier with the Langchain tasks.

In [ ]:
# llm = HuggingFacePipeline(pipeline=query_pipeline)

# # checking again that everything is working fine
# time_start = time()
# question = "Please explain what EU AI Act is."
# response = llm(prompt=question)
# time_end = time()
# total_time = f"{round(time_end-time_start, 3)} sec."
# full_response =  f"Question: {question}\nAnswer: {response}\nTotal time: {total_time}"
# display(Markdown(colorize_text(full_response)))

In [25]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

time_start =time()
question = "Please explain what EU AI Act is."
response= llm(prompt=question)
time_end = time()

total_time = f"{round(time_end-time_start, 3)} sec."

full_response = f"Question: {question}\nAnswer: {response}\nTotal Time: {total_time}"

display(Markdown(colorize_text(full_response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: Please explain what EU AI Act is.
Answer:  What are its main goals and objectives? What are the implications of this act for the tech industry and society at large?
The EU AI Act, officially known as the Regulation on a Framework for the Conditions of High-Performance Computing and Artificial Intelligence (EU AI Act), is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

Main goals and objectives:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy, transparent, and explainable, with a focus on preventing biases and ensuring fairness.
2. **Risk management**: The regulation seeks to identify and mitigate risks associated with AI, such as job displacement, privacy concerns, and potential biases.
3. **Standardization**: The EU AI Act aims to establish common standards for AI development, deployment, and use, ensuring consistency across the EU.
4. **Accountability**: The regulation requires AI developers and users to be accountable for the consequences of their AI systems.

Implications for the tech industry and society:

1. **Increased transparency**: The EU AI Act will require AI developers to provide clear explanations of their AI systems' decision-making processes, increasing transparency and accountability.
2. **Responsible innovation**: The regulation will encourage responsible innovation in AI, promoting the development of AI systems that are fair, transparent, and explainable.
3. **Job displacement concerns**: The EU AI Act may lead to increased job displacement concerns, as AI systems may replace certain jobs, potentially exacerbating existing social and economic inequalities.
4. **Data protection**: The regulation will strengthen data protection rules, ensuring that AI systems handle personal data responsibly and in accordance with EU data protection regulations.
5. **Increased costs**: The EU AI Act may lead to increased costs for AI developers and users, as they will need to comply with the regulation's requirements and implement necessary measures to ensure trustworthy AI development and deployment.

Overall, the EU AI Act aims to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values. While it may present challenges for the tech industry, it is an important step towards ensuring that AI benefits society as a whole.assistant

The EU AI Act is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

The main goals and objectives of the EU AI Act are:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy, transparent, and explainable, with a focus on preventing biases and ensuring fairness.
2. **Risk management**: The regulation seeks to identify and mitigate risks associated with AI, such as job displacement, privacy concerns, and potential biases.
3. **Standardization**: The EU AI Act aims to establish common standards for AI development, deployment, and use, ensuring consistency across the EU.
4. **Accountability**: The regulation requires AI developers and users to be accountable for the consequences of their AI systems.

The implications of the EU AI Act for the tech industry and society are:

1. **Increased transparency**: The EU AI Act will require AI developers to provide clear explanations of their AI systems' decision-making processes, increasing transparency and accountability.
2. **Responsible innovation**: The regulation will encourage responsible innovation in AI, promoting the development of AI systems that are fair, transparent, and explainable.
3. **Job displacement concerns**: The EU AI Act may lead to increased job displacement concerns, as AI systems may replace certain jobs, potentially exacerbating existing social and economic inequalities.
4. **Data protection**: The regulation will strengthen data protection rules, ensuring that AI systems handle personal data responsibly and in accordance with EU data protection regulations.
5. **Increased costs**: The EU AI Act may lead to increased costs for AI developers and users, as they will need to comply with the regulation's requirements and implement necessary measures to ensure trustworthy AI development and deployment.

Overall, the EU AI Act aims to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values. While it may present challenges for the tech industry, it is an important step towards ensuring that AI benefits society as a whole.assistant

The EU AI Act is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

The main goals and objectives of the EU AI Act are:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy,
Total Time: 89.394 sec.

## Ingestion of data using PyPDFLoader

We will ingest the EU AI Act data using the PyPDFLoader from Langchain. There are actually multiple PDF ingestion utilities, we selected this one since it is easy to use.

In [26]:
loader = PyPDFLoader("/kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf")
documents = loader.load()

In [27]:
documents[0]

Document(page_content=' \n \n \n5662/24\n \n \n \nRB/ek\n \n1\n \n \nTREE.2.B\n \nLIMITE\n \nEN\n \n \n \n Council of the  \nEuropean Union    \n \n \n \n \n \nBrussels , 26 January  2024  \n(OR. en) \n \n \n5662 /24 \n \n \nLIMITE  \n \nTELECOM 22  \nJAI 98  \nCOPEN 18  \nCYBER 14  \nDATAPROTECT 32  \nEJUSTICE 3  \nCOSI 6  \nIXIM 15  \nENFOPOL 21  \nRELEX 77  \nMI 65  \nCOMPET 68  \nCODEC 1 33  \n \nInterinstitutional File:  \n2021/0106 (COD)   \n  \n \nNOTE  \nFrom:  Presidency  \nTo: Permanent Representatives Committee  \nNo. Cion doc.:  8115/21  \nSubject:  Proposal for a Regulation of the European Parliament and of the Council \nlaying down harmonised rules on artificial intelligence (Artificial Inte lligence \nAct)  and amending certain Union legislative acts  \n- Analysis of the final compromise text with a view t o agreement  \n \n \nI.\n \nINTRODUCTION\n \n1.\n \nThe Commission adopted the proposal for a Regulation laying down harmonised rules on \nartificial intelligence (Art

In [28]:
documents[3]

Document(page_content=' \n \n \n5662/24\n \n \n \nRB/ek\n \n4\n \n \nTREE.2.B\n \nLIMITE\n \nEN\n \n \nhave been added to \nAnnex III\n \n(List of high\n-\nrisk AI\n \nsystems), with some further limitations \nof their scope.\n \n \n4.\n \nPost\n-\nremote biometric identification\n \n \nIn addition to being included in the list of high\n-\nrisk AI systems in \nAnnex III\n, the use of \npost\n-\nremote biometric identification by law enforcement authorities has been made subject \nto some additional safeguards in \nArticle 29\n. These safeguards include some limited \ntransparency measures and the requirement for law enforcement authorit\nies to request an \nauthorisation for the use of post\n-\nremote biometric identification, which could be given by a \njudicial authority or an administrative authority whose decision is binding and subject to a \njudicial review. This can be done prior or ex post, \nwithin 48 hours. Furthermore, the \ncompromise text clarifies that post\n-\nremote bio

## Split data in chunks

We split data in chunks using a recursive character text splitter.  

Note: You can experiment with several values of chunk_size and chunk_overlap. Here we will set the following values:
* chunk_size: 1000 (this gives the size of a chunk, in characters). 
* chunk_overlap: 100 (this gives the number of characters with which two succesive chunks overlap).  

Chunk overlapping is required in order to be able to keep the context, even if we have a concept that we want to include that is spread over multiple document chunks.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

In [30]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

In [31]:
all_splits[3]

Document(page_content='Following this political agreement, the co\n-\nlegislators proceeded to carry out further technical \nwork in January 2024 in order to align the text of the recitals with the text of the articles as \nagreed during the final trilogue. \n \n \n5.\n \nIn the Annex t\no this document delegations will find the text of the proposal for a \nRegulation \nlaying down harmonised rules on artificial intelligence (the AI Act)\n, updated according to the \nprovisional political agreement reached at the fifth trilogue between 6 and 8 Dec\nember 2023\n.\n \n \nII.\n \nMAIN ELEMENTS OF THE COMPROMISE\n \n1.\n \nSubject matter and scope\n \n \n \nAs regards the subject matter, the compromise text of \nArticle 1(1)\n \nincludes a high\n-\nlevel \nstatement that one of the purposes of the AI Act is to ensure a high level of protection of \nhealth, safety and fundamental rights enshrined in the Charter, which includes democracy, \nrule of law and environmental protection. However, 

## Creating Embeddings and Storing in Vector Store

Create the embeddings using Sentence Transformer and HuggingFace embeddings.  
Ocasionally, HuggingFace sentence-transformers might not be available. We implement therefore a mechanism to work with local stored sentence transformers.

In [32]:

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

# try to access the sentence transformers from HuggingFace: https://huggingface.co/api/models/sentence-transformers/all-mpnet-base-v2
try:
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
except Exception as ex:
    print("Exception: ", ex)
    # alternatively, we will access the embeddings models locally
    local_model_path = "/kaggle/input/sentence-transformers/minilm-l6-v2/all-MiniLM-L6-v2"
    print(f"Use alternative (local) model: {local_model_path}\n")
    embeddings = HuggingFaceEmbeddings(model_name=local_model_path, model_kwargs=model_kwargs)

Initialize ChromaDB with the document splits, the embeddings defined previously and with the option to persist it locally.  
We make sure to use the persistence option for the vector database.

In [39]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

In [40]:
vectordb.__getattribute__

<method-wrapper '__getattribute__' of Chroma object at 0x784981f6c670>

## Initialize chain   

We are using `RetrievalQA` task chain utility from Langchain.  
This will first query the vector database (using similarity search) with the prompt we are using.   
Then, the query and the context retrieved (the documents that match with the query) are used to compose a prompt that instructs the LLM to answer to the query (**Generation**) using the information from the context retrieved (**Retrieval**). Therefore the name of the system, `Retrieval Augmented Generation`. 


In [43]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = retriever,
    verbose = True

)

In [38]:
# retriever = vectordb.as_retriever()

# qa = RetrievalQA.from_chain_type(
#     llm=llm, 
#     chain_type="stuff", 
#     retriever=retriever, 
#     verbose=True
# )

AttributeError: 'coroutine' object has no attribute 'as_retriever'

## Test the Retrieval-Augmented Generation 


We define a test function, that will run the query and time it.

In [ ]:
def test_rag(qa, query):

    time_start = time()
    response = qa.run(query)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."

    full_response =  f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))

In [46]:
def test_rag(qa, query):
    
    time_start = time()
    response = qa.run(query)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."
    
    full_repsonse = f"question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))

Let's check few queries.

In [50]:
query = "How is performed the testing of high-risk AI systems in real world conditions?"
test_rag(qa, query)



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.




**<font color='red'>Question:</font>** Please explain what EU AI Act is.


**<font color='green'>Answer:</font>**  What are its main goals and objectives? What are the implications of this act for the tech industry and society at large?
The EU AI Act, officially known as the Regulation on a Framework for the Conditions of High-Performance Computing and Artificial Intelligence (EU AI Act), is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

Main goals and objectives:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy, transparent, and explainable, with a focus on preventing biases and ensuring fairness.
2. **Risk management**: The regulation seeks to identify and mitigate risks associated with AI, such as job displacement, privacy concerns, and potential biases.
3. **Standardization**: The EU AI Act aims to establish common standards for AI development, deployment, and use, ensuring consistency across the EU.
4. **Accountability**: The regulation requires AI developers and users to be accountable for the consequences of their AI systems.

Implications for the tech industry and society:

1. **Increased transparency**: The EU AI Act will require AI developers to provide clear explanations of their AI systems' decision-making processes, increasing transparency and accountability.
2. **Responsible innovation**: The regulation will encourage responsible innovation in AI, promoting the development of AI systems that are fair, transparent, and explainable.
3. **Job displacement concerns**: The EU AI Act may lead to increased job displacement concerns, as AI systems may replace certain jobs, potentially exacerbating existing social and economic inequalities.
4. **Data protection**: The regulation will strengthen data protection rules, ensuring that AI systems handle personal data responsibly and in accordance with EU data protection regulations.
5. **Increased costs**: The EU AI Act may lead to increased costs for AI developers and users, as they will need to comply with the regulation's requirements and implement necessary measures to ensure trustworthy AI development and deployment.

Overall, the EU AI Act aims to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values. While it may present challenges for the tech industry, it is an important step towards ensuring that AI benefits society as a whole.assistant

The EU AI Act is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

The main goals and objectives of the EU AI Act are:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy, transparent, and explainable, with a focus on preventing biases and ensuring fairness.
2. **Risk management**: The regulation seeks to identify and mitigate risks associated with AI, such as job displacement, privacy concerns, and potential biases.
3. **Standardization**: The EU AI Act aims to establish common standards for AI development, deployment, and use, ensuring consistency across the EU.
4. **Accountability**: The regulation requires AI developers and users to be accountable for the consequences of their AI systems.

The implications of the EU AI Act for the tech industry and society are:

1. **Increased transparency**: The EU AI Act will require AI developers to provide clear explanations of their AI systems' decision-making processes, increasing transparency and accountability.
2. **Responsible innovation**: The regulation will encourage responsible innovation in AI, promoting the development of AI systems that are fair, transparent, and explainable.
3. **Job displacement concerns**: The EU AI Act may lead to increased job displacement concerns, as AI systems may replace certain jobs, potentially exacerbating existing social and economic inequalities.
4. **Data protection**: The regulation will strengthen data protection rules, ensuring that AI systems handle personal data responsibly and in accordance with EU data protection regulations.
5. **Increased costs**: The EU AI Act may lead to increased costs for AI developers and users, as they will need to comply with the regulation's requirements and implement necessary measures to ensure trustworthy AI development and deployment.

Overall, the EU AI Act aims to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values. While it may present challenges for the tech industry, it is an important step towards ensuring that AI benefits society as a whole.assistant

The EU AI Act is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

The main goals and objectives of the EU AI Act are:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy,
Total Time: 89.394 sec.

In [51]:
query = "What are the operational obligations of notified bodies?"
test_rag(qa, query)



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.




**<font color='red'>Question:</font>** Please explain what EU AI Act is.


**<font color='green'>Answer:</font>**  What are its main goals and objectives? What are the implications of this act for the tech industry and society at large?
The EU AI Act, officially known as the Regulation on a Framework for the Conditions of High-Performance Computing and Artificial Intelligence (EU AI Act), is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

Main goals and objectives:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy, transparent, and explainable, with a focus on preventing biases and ensuring fairness.
2. **Risk management**: The regulation seeks to identify and mitigate risks associated with AI, such as job displacement, privacy concerns, and potential biases.
3. **Standardization**: The EU AI Act aims to establish common standards for AI development, deployment, and use, ensuring consistency across the EU.
4. **Accountability**: The regulation requires AI developers and users to be accountable for the consequences of their AI systems.

Implications for the tech industry and society:

1. **Increased transparency**: The EU AI Act will require AI developers to provide clear explanations of their AI systems' decision-making processes, increasing transparency and accountability.
2. **Responsible innovation**: The regulation will encourage responsible innovation in AI, promoting the development of AI systems that are fair, transparent, and explainable.
3. **Job displacement concerns**: The EU AI Act may lead to increased job displacement concerns, as AI systems may replace certain jobs, potentially exacerbating existing social and economic inequalities.
4. **Data protection**: The regulation will strengthen data protection rules, ensuring that AI systems handle personal data responsibly and in accordance with EU data protection regulations.
5. **Increased costs**: The EU AI Act may lead to increased costs for AI developers and users, as they will need to comply with the regulation's requirements and implement necessary measures to ensure trustworthy AI development and deployment.

Overall, the EU AI Act aims to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values. While it may present challenges for the tech industry, it is an important step towards ensuring that AI benefits society as a whole.assistant

The EU AI Act is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

The main goals and objectives of the EU AI Act are:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy, transparent, and explainable, with a focus on preventing biases and ensuring fairness.
2. **Risk management**: The regulation seeks to identify and mitigate risks associated with AI, such as job displacement, privacy concerns, and potential biases.
3. **Standardization**: The EU AI Act aims to establish common standards for AI development, deployment, and use, ensuring consistency across the EU.
4. **Accountability**: The regulation requires AI developers and users to be accountable for the consequences of their AI systems.

The implications of the EU AI Act for the tech industry and society are:

1. **Increased transparency**: The EU AI Act will require AI developers to provide clear explanations of their AI systems' decision-making processes, increasing transparency and accountability.
2. **Responsible innovation**: The regulation will encourage responsible innovation in AI, promoting the development of AI systems that are fair, transparent, and explainable.
3. **Job displacement concerns**: The EU AI Act may lead to increased job displacement concerns, as AI systems may replace certain jobs, potentially exacerbating existing social and economic inequalities.
4. **Data protection**: The regulation will strengthen data protection rules, ensuring that AI systems handle personal data responsibly and in accordance with EU data protection regulations.
5. **Increased costs**: The EU AI Act may lead to increased costs for AI developers and users, as they will need to comply with the regulation's requirements and implement necessary measures to ensure trustworthy AI development and deployment.

Overall, the EU AI Act aims to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values. While it may present challenges for the tech industry, it is an important step towards ensuring that AI benefits society as a whole.assistant

The EU AI Act is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

The main goals and objectives of the EU AI Act are:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy,
Total Time: 89.394 sec.

In [52]:
query = "What are the unacceptable risks?"
test_rag(qa, query)



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.




**<font color='red'>Question:</font>** Please explain what EU AI Act is.


**<font color='green'>Answer:</font>**  What are its main goals and objectives? What are the implications of this act for the tech industry and society at large?
The EU AI Act, officially known as the Regulation on a Framework for the Conditions of High-Performance Computing and Artificial Intelligence (EU AI Act), is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

Main goals and objectives:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy, transparent, and explainable, with a focus on preventing biases and ensuring fairness.
2. **Risk management**: The regulation seeks to identify and mitigate risks associated with AI, such as job displacement, privacy concerns, and potential biases.
3. **Standardization**: The EU AI Act aims to establish common standards for AI development, deployment, and use, ensuring consistency across the EU.
4. **Accountability**: The regulation requires AI developers and users to be accountable for the consequences of their AI systems.

Implications for the tech industry and society:

1. **Increased transparency**: The EU AI Act will require AI developers to provide clear explanations of their AI systems' decision-making processes, increasing transparency and accountability.
2. **Responsible innovation**: The regulation will encourage responsible innovation in AI, promoting the development of AI systems that are fair, transparent, and explainable.
3. **Job displacement concerns**: The EU AI Act may lead to increased job displacement concerns, as AI systems may replace certain jobs, potentially exacerbating existing social and economic inequalities.
4. **Data protection**: The regulation will strengthen data protection rules, ensuring that AI systems handle personal data responsibly and in accordance with EU data protection regulations.
5. **Increased costs**: The EU AI Act may lead to increased costs for AI developers and users, as they will need to comply with the regulation's requirements and implement necessary measures to ensure trustworthy AI development and deployment.

Overall, the EU AI Act aims to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values. While it may present challenges for the tech industry, it is an important step towards ensuring that AI benefits society as a whole.assistant

The EU AI Act is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

The main goals and objectives of the EU AI Act are:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy, transparent, and explainable, with a focus on preventing biases and ensuring fairness.
2. **Risk management**: The regulation seeks to identify and mitigate risks associated with AI, such as job displacement, privacy concerns, and potential biases.
3. **Standardization**: The EU AI Act aims to establish common standards for AI development, deployment, and use, ensuring consistency across the EU.
4. **Accountability**: The regulation requires AI developers and users to be accountable for the consequences of their AI systems.

The implications of the EU AI Act for the tech industry and society are:

1. **Increased transparency**: The EU AI Act will require AI developers to provide clear explanations of their AI systems' decision-making processes, increasing transparency and accountability.
2. **Responsible innovation**: The regulation will encourage responsible innovation in AI, promoting the development of AI systems that are fair, transparent, and explainable.
3. **Job displacement concerns**: The EU AI Act may lead to increased job displacement concerns, as AI systems may replace certain jobs, potentially exacerbating existing social and economic inequalities.
4. **Data protection**: The regulation will strengthen data protection rules, ensuring that AI systems handle personal data responsibly and in accordance with EU data protection regulations.
5. **Increased costs**: The EU AI Act may lead to increased costs for AI developers and users, as they will need to comply with the regulation's requirements and implement necessary measures to ensure trustworthy AI development and deployment.

Overall, the EU AI Act aims to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values. While it may present challenges for the tech industry, it is an important step towards ensuring that AI benefits society as a whole.assistant

The EU AI Act is a proposed regulation aimed at ensuring the trustworthy development, deployment, and use of Artificial Intelligence (AI) in the European Union. The act is part of the EU's efforts to promote the safe and responsible development of AI, while also ensuring the protection of citizens' rights and fundamental values.

The main goals and objectives of the EU AI Act are:

1. **Trustworthy AI**: The EU AI Act aims to ensure that AI systems are trustworthy,
Total Time: 89.394 sec.

## Document sources

Let's check the documents sources, for the last query run.  


In [54]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: What are the unacceptable risks?
Retrieved documents: 4
Source:  /kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf
Text:  case of the materialisation of these risk
s
. The impact assessment should apply to the first 

Source:  /kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf
Text:  reasonably foreseeable misuse, which may lead to risks to the health and safety 
or fundamental rights referred to in Article 9(2); 

Source:  /kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf
Text:  potential to remove guardrails and other factors. In particular, international approaches 
have so far identified the need to devote attention to risks from potential intentional misuse 
or unintended issues of control relating t
o alignment with human intent; chemical, 
biological, radiological, and nuclear risks, such as the ways in which barriers to entry can 
be lowered, including for weapons development, design acquisition, or use; offensive 
cyber capabilities, such as

/tmp/ipykernel_33/641566907.py:1: RuntimeWarning: coroutine 'VectorStore.asimilarity_search' was never awaited
  docs = vectordb.similarity_search(query)


In [58]:
# docs = vectordb.similarity_search(query)
# print(f"Query: {query}")
# print(f"Retrieved documents: {len(docs)}")

# for doc in docs:
#     doc_details = doc.to_json()['kwargs']
#     print("source:", doc_details['metadata']['source'])
#     print("Text", doc_details['page_content'], "\n")

# Conclusions


We used Langchain, ChromaDB and Llama3 as a LLM to build a Retrieval Augmented Generation solution. For testing, we were using the EU AI Act from 2023.  
The answers to questions from EU AI Act are correct, when using a RAG model.  

To improve the solution, we will have to refine the RAG implementation, first by optimizing the embeddings, then by using more complex RAG schemes.



